In [134]:
from sklearn.metrics import auc, roc_auc_score, roc_curve, recall_score, precision_score
import warnings
from itertools import combinations
from sklearn.metrics import confusion_matrix, cohen_kappa_score, f1_score, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, MissingIndicator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pandas import concat
from datetime import datetime, timedelta
%matplotlib inline


warnings.filterwarnings('ignore')

In [135]:
# Зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы:
RANDOM_SEED = 41

# Предобработка

In [136]:
def update_produv(produv_train):
    
    produv_start=pd.pivot_table(produv_train,index='NPLV',values='SEC', aggfunc={'min'}).reset_index()
    produv_end=pd.pivot_table(produv_train,index='NPLV',values='SEC', aggfunc={'max'}).reset_index()
    ras_med=pd.pivot_table(produv_train,index='NPLV',values='RAS', aggfunc={'mean'}).reset_index()
    ras_sum=pd.pivot_table(produv_train,index='NPLV',values='RAS', aggfunc={'sum'}).reset_index()
    pol_start=pd.pivot_table(produv_train,index='NPLV',values='POL', aggfunc={'max'}).reset_index()
    pol_end=pd.pivot_table(produv_train,index='NPLV',values='POL', aggfunc={'min'}).reset_index()
    
    produv_upd=pd.merge(produv_start,produv_end, how='left',on='NPLV')
    produv_upd=pd.merge(produv_upd,ras_sum, how='left',on='NPLV')
    produv_upd=pd.merge(produv_upd,pol_start, how='left',on='NPLV')
    produv_upd=pd.merge(produv_upd,pol_end, how='left',on='NPLV')
    
    #df.rename(columns = {list(df)[1]:'new_name'}, inplace=True)
    
    produv_upd.rename(columns={list(produv_upd)[1]: 'PRODUVKA_START'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[2]: 'PRODUVKA_END'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[3]: 'RASHOD_SUM'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[4]: 'POL_MAX'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[5]: 'POL_MIN'}, inplace=True)
    
    produv_upd['PRODUVKA_TIME']=produv_upd['PRODUVKA_END']-produv_upd['PRODUVKA_START']
    produv_upd['PRODUVKA_TIME,SEC']=produv_upd['PRODUVKA_TIME'].dt.seconds

    
    return produv_upd


def update_gas(gas_train):
    V_finish=gas_train[['NPLV','Time','V']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    V_finish=V_finish.drop('Time', axis=1)
    V_start=gas_train[['NPLV','Time','V']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    V_start=V_start.drop('Time', axis=1)
    T_finish=gas_train[['NPLV','Time','T']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    T_finish=T_finish.drop('Time', axis=1)
    T_start=gas_train[['NPLV','Time','T']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    T_start=T_start.drop('Time', axis=1)
    O2_sum=pd.pivot_table(gas_train,index='NPLV',values='O2', aggfunc={'sum'}).reset_index()
    N2_sum=pd.pivot_table(gas_train,index='NPLV',values='N2', aggfunc={'sum'}).reset_index()
    H2_sum=pd.pivot_table(gas_train,index='NPLV',values='H2', aggfunc={'sum'}).reset_index()
    CO2_sum=pd.pivot_table(gas_train,index='NPLV',values='CO2', aggfunc={'sum'}).reset_index()
    CO_sum=pd.pivot_table(gas_train,index='NPLV',values='CO', aggfunc={'sum'}).reset_index()
    AR_sum=pd.pivot_table(gas_train,index='NPLV',values='AR', aggfunc={'sum'}).reset_index()
    TF1_start=gas_train[['NPLV','Time','T фурмы 1']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    TF1_start=TF1_start.drop('Time', axis=1)
    TF1_end=gas_train[['NPLV','Time','T фурмы 1']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    TF1_end=TF1_end.drop('Time', axis=1)
    TF2_start=gas_train[['NPLV','Time','T фурмы 2']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    TF2_start=TF2_start.drop('Time', axis=1)
    TF2_end=gas_train[['NPLV','Time','T фурмы 2']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    TF2_end=TF2_end.drop('Time', axis=1)
    O2_pressure_start=gas_train[['NPLV','Time','O2_pressure']].sort_values(by=['NPLV','Time'], ascending=[True,True]).drop_duplicates('NPLV')
    O2_pressure_start=O2_pressure_start.drop('Time', axis=1)
    O2_pressure_end=gas_train[['NPLV','Time','O2_pressure']].sort_values(by=['NPLV','Time'], ascending=[True,False]).drop_duplicates('NPLV')
    O2_pressure_end=O2_pressure_end.drop('Time', axis=1)
    O2_pressure_max=pd.pivot_table(gas_train,index='NPLV',values='O2_pressure', aggfunc={'max'}).reset_index()
    O2_pressure_min=pd.pivot_table(gas_train,index='NPLV',values='O2_pressure', aggfunc={'min'}).reset_index()
    time_start=pd.pivot_table(gas_train,index='NPLV',values='Time', aggfunc={'min'}).reset_index()
    time_end=pd.pivot_table(gas_train,index='NPLV',values='Time', aggfunc={'max'}).reset_index()
    tmp_1=pd.merge(time_start, time_end, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, V_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, V_finish, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, T_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, T_finish, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, N2_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, H2_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, CO2_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, CO_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, AR_sum, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, TF1_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, TF1_end, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, TF2_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, TF2_end, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_pressure_start, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_pressure_end, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_pressure_max, how='left', on='NPLV')
    tmp_1=pd.merge(tmp_1, O2_pressure_min, how='left', on='NPLV')
    tmp_1.rename(columns={list(tmp_1)[1]:'GAS_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[2]: 'GAS_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[3]: 'V_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[4]: 'V_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[5]: 'T_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[6]: 'T_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[7]: 'O2_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[8]: 'N2_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[9]: 'H2_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[10]: 'CO2_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[11]: 'CO_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[12]: 'AR_SUM'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[13]: 'TF1_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[14]: 'TF1_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[15]: 'TF2_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[16]: 'TF2_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[17]: 'O2_PRESSURE_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[18]: 'O2_PRESSURE_FINISH'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[19]: 'O2_PRESSURE_MAX'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[20]: 'O2_PRESSURE_MIN'}, inplace=True)
    tmp_1['GAS_TIME']=tmp_1['GAS_FINISH']-tmp_1['GAS_START']
    tmp_1['GAS_TIME,SEC']=tmp_1['GAS_TIME'].dt.seconds
    return tmp_1


def update_sip(sip_train):
    sip_time_start=sip_train[['NPLV','DAT_OTD']].sort_values(by=['NPLV','DAT_OTD'], ascending=[True,True]).drop_duplicates('NPLV')
    sip_time_finish=sip_train[['NPLV','DAT_OTD']].sort_values(by=['NPLV','DAT_OTD'], ascending=[True,False]).drop_duplicates('NPLV')
    sip_sostav=pd.pivot_table(sip_train,index='NPLV',columns='VDSYP',values='VSSYP', aggfunc={'sum'})
    tmp_1=pd.merge(sip_sostav, sip_time_start, how='left',on='NPLV')
    tmp_1=pd.merge(tmp_1, sip_time_finish, how='left',on='NPLV')
    tmp_1.rename(columns={list(tmp_1)[1]:'TOTAL_104'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[2]:'TOTAL_119'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[3]:'TOTAL_171'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[4]:'TOTAL_346'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[5]:'TOTAL_397'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[6]:'TOTAL_408'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[7]:'TOTAL_442'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[8]:'SIP_START'}, inplace=True)
    tmp_1.rename(columns={list(tmp_1)[9]:'SIP_FINISH'}, inplace=True)
    tmp_1['SIP_TIME']=tmp_1['SIP_FINISH']-tmp_1['SIP_START']
    tmp_1['SIP_TIME,SEC']=tmp_1['SIP_TIME'].dt.seconds
    return tmp_1

def update_chronom(df):
    
    O2_sum=pd.pivot_table(df,index='NPLV',values='O2', aggfunc={'sum'}).reset_index()
        
    O2_sum.rename(columns={list(O2_sum)[1]: 'O2_SUM'}, inplace=True)      
   
    return O2_sum


def update_lom(lom):
    
    vdl_sum=pd.pivot_table(lom,index='NPLV',values='VDL', aggfunc={'sum'}).reset_index()
    lom_sum=pd.pivot_table(lom,index='NPLV',values='VES', aggfunc={'sum'}).reset_index()
    
    lom_upd=pd.merge(vdl_sum, lom_sum, how='left',on='NPLV')
        
    #df.rename(columns = {list(df)[1]:'new_name'}, inplace=True)
    
    lom_upd.rename(columns={list(lom_upd)[1]: 'VDL_SUM'}, inplace=True)
    lom_upd.rename(columns={list(lom_upd)[2]: 'LOM_SUM'}, inplace=True)
        
    return lom_upd


def duplicate_data(df):
    '''
    Функция для поиска дубликатов.
    '''
    if len(df) > len(df.drop_duplicates()):
        print('Имеются дубликаты')
        display(df[df.duplicated()])
    else:
        print('Дубликатов не обнаружено')

In [137]:
path= 'C:/Users/Артемий/Python/EVRAZ_AI/data_task1/'

names=['chronom','chugun','gas','lom','plavki','produv','sip']

chrom_train=pd.read_csv(path+names[0]+'_train.csv')
chugun_train=pd.read_csv(path+names[1]+'_train.csv')
gas_train=pd.read_csv(path+names[2]+'_train.csv')
lom_train=pd.read_csv(path+names[3]+'_train.csv')
plavki_train=pd.read_csv(path+names[4]+'_train.csv')
produv_train=pd.read_csv(path+names[5]+'_train.csv')
sip_train=pd.read_csv(path+names[6]+'_train.csv')

chrom_test=pd.read_csv(path+names[0]+'_test.csv')
chugun_test=pd.read_csv(path+names[1]+'_test.csv')
gas_test=pd.read_csv(path+names[2]+'_test.csv')
lom_test=pd.read_csv(path+names[3]+'_test.csv')
plavki_test=pd.read_csv(path+names[4]+'_test.csv')
produv_test=pd.read_csv(path+names[5]+'_test.csv')
sip_test=pd.read_csv(path+names[6]+'_test.csv')

target_train=pd.read_csv(path+'target_train.csv')

In [138]:
gas_train['Time']=pd.to_datetime(gas_train['Time'], format='%Y-%m-%d %H:%M:%S')
gas_upd=update_gas(gas_train)
gas_upd

,NPLV,GAS_START,GAS_FINISH,V_START,V_FINISH,T_START,T_FINISH,CO_SUM,AR_SUM,CO_SUM,...,TF1_START,TF1_FINISH,TF2_START,TF2_FINISH,O2_PRESSURE_START,O2_PRESSURE_FINISH,O2_PRESSURE_MAX,O2_PRESSURE_MIN,GAS_TIME,"GAS_TIME,SEC"
0,510008,2021-01-01 03:08:11.437,2021-01-01 03:51:10.437,218263.34375,219321.343750,262.847229,131.944443,20685.819848,156085.786997,768.890272,...,0.000000,0.000000,0.000000,0.000000,13.085938,13.389757,14.149305,12.594040,0 days 00:42:59,2579
1,510009,2021-01-01 04:00:44.437,2021-01-01 05:07:26.437,219321.34375,218792.984375,200.000000,114.236115,43381.103374,255833.503217,644.923314,...,0.000000,0.000000,0.000000,0.000000,12.782118,13.628472,15.711805,11.971932,0 days 01:06:42,4002
2,510010,2021-01-01 05:12:29.437,2021-01-01 06:00:52.437,215810.28125,216986.937500,211.805557,127.430550,25108.381611,179821.062543,898.578333,...,0.000000,0.000000,0.000000,0.000000,14.301215,13.823785,14.829283,13.158276,0 days 00:48:23,2903
3,510011,2021-01-01 06:13:47.437,2021-01-01 07:08:38.437,218263.34375,221631.140625,131.944443,112.847221,29401.916247,204155.891649,327.293705,...,0.000000,0.000000,0.000000,0.000000,13.577835,14.626736,15.704573,12.905092,0 days 00:54:51,3291
4,510012,2021-01-01 07:13:45.437,2021-01-01 08:01:59.437,216238.87500,219532.343750,250.694443,133.333328,24225.586513,178139.248331,1266.473051,...,0.000000,0.000000,0.000000,0.000000,14.402488,15.567130,15.567130,13.266783,0 days 00:48:14,2894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,512318,2021-04-26 13:04:25.437,2021-04-26 13:55:49.437,207498.31250,206042.953125,187.152786,131.597214,12707.545640,163760.137200,3647.162277,...,25.056086,24.276621,25.781250,26.060578,16.066261,16.109484,16.268808,13.122108,0 days 00:51:24,3084
2059,512319,2021-04-26 14:10:20.437,2021-04-26 15:14:22.437,208499.93750,213545.890625,112.847221,134.195474,31294.109473,237604.199027,2650.162875,...,24.726665,24.019264,26.225303,26.042221,16.095198,14.351852,16.102430,12.991898,0 days 01:04:02,3842
2060,512320,2021-04-26 15:21:37.437,2021-04-26 16:16:42.437,209717.62500,206379.718750,272.222229,123.611115,22498.203495,185790.109712,3335.450657,...,24.479168,24.645240,26.069058,26.475694,15.111401,15.509260,16.276043,13.809318,0 days 00:55:05,3305
2061,512321,2021-04-26 16:22:37.437,2021-04-26 17:23:36.437,205367.78125,207721.328125,248.958328,103.819443,21993.862756,198195.086008,3517.636915,...,25.231480,24.117824,26.475691,27.624052,16.247107,13.143807,16.290510,13.143807,0 days 01:00:59,3659


In [139]:
sip_train['DAT_OTD']=pd.to_datetime(sip_train['DAT_OTD'], format='%Y-%m-%d %H:%M:%S')
sip_upd=update_sip(sip_train)
sip_upd

,NPLV,TOTAL_104,TOTAL_119,TOTAL_171,TOTAL_346,TOTAL_397,TOTAL_408,TOTAL_442,SIP_START,SIP_FINISH,SIP_TIME,"SIP_TIME,SEC"
0,510008,NaN,NaN,980.0,2950.0,NaN,14080.0,2960.0,2021-01-01 03:03:53,2021-01-01 03:33:48,0 days 00:29:55,1795
1,510009,1060.0,NaN,960.0,2930.0,NaN,18830.0,NaN,2021-01-01 03:55:43,2021-01-01 04:40:13,0 days 00:44:30,2670
2,510010,990.0,NaN,1050.0,2990.0,NaN,16080.0,2960.0,2021-01-01 05:09:20,2021-01-01 05:40:52,0 days 00:31:32,1892
3,510011,550.0,NaN,980.0,3620.0,NaN,22150.0,NaN,2021-01-01 06:01:36,2021-01-01 06:56:40,0 days 00:55:04,3304
4,510012,2050.0,NaN,1000.0,2930.0,NaN,19550.0,3010.0,2021-01-01 07:11:44,2021-01-01 07:50:19,0 days 00:38:35,2315
...,...,...,...,...,...,...,...,...,...,...,...,...
2058,512318,NaN,NaN,NaN,NaN,NaN,14640.0,2890.0,2021-04-26 12:58:37,2021-04-26 13:45:32,0 days 00:46:55,2815
2059,512319,NaN,NaN,NaN,960.0,NaN,19210.0,2820.0,2021-04-26 13:57:24,2021-04-26 15:03:11,0 days 01:05:47,3947
2060,512320,NaN,NaN,NaN,1380.0,NaN,19420.0,2830.0,2021-04-26 15:17:30,2021-04-26 16:05:55,0 days 00:48:25,2905
2061,512321,NaN,NaN,NaN,520.0,NaN,16100.0,2310.0,2021-04-26 16:19:13,2021-04-26 16:52:48,0 days 00:33:35,2015


In [140]:
chronom_train['VR_NACH']=pd.to_datetime(chronom_train['VR_NACH'], format='%Y-%m-%d %H:%M:%S')
chronom_train = chronom_train[chronom_train['VR_NACH'].dt.year > 2020]
chronom_upd = update_chronom(chronom_train)
chronom_upd

,NPLV,O2_SUM
0,510008,2909.0
1,510009,2182.0
2,510010,2841.0
3,510011,10.0
4,510012,3225.0
...,...,...
2058,512318,1917.0
2059,512319,1660.0
2060,512320,0.0
2061,512321,371.0


In [141]:
produv_train['SEC']=pd.to_datetime(produv_train['SEC'], format='%Y-%m-%d %H:%M:%S')
produv_upd=update_produv(produv_train)
produv_upd

,NPLV,PRODUVKA_START,PRODUVKA_END,RASHOD_SUM,POL_MAX,POL_MIN,PRODUVKA_TIME,"PRODUVKA_TIME,SEC"
0,510008,2021-01-01 03:18:26,2021-01-01 04:26:54,838814.0,7.070000,0.77,0 days 01:08:28,4108
1,510009,2021-01-01 04:26:56,2021-01-01 05:25:08,764750.0,4.760000,0.63,0 days 00:58:12,3492
2,510010,2021-01-01 05:25:10,2021-01-01 06:23:50,764170.5,4.349765,0.65,0 days 00:58:40,3520
3,510011,2021-01-01 06:23:52,2021-01-01 07:26:04,733717.0,12.490000,0.79,0 days 01:02:12,3732
4,510012,2021-01-01 07:26:06,2021-01-01 08:36:44,840471.0,5.000000,0.67,0 days 01:10:38,4238
...,...,...,...,...,...,...,...,...
2058,512318,2021-04-26 13:10:12,2021-04-26 14:37:38,978520.0,10.100000,0.80,0 days 01:27:26,5246
2059,512319,2021-04-26 14:37:40,2021-04-26 15:35:46,783447.0,10.040000,0.83,0 days 00:58:06,3486
2060,512320,2021-04-26 15:35:48,2021-04-26 16:36:02,742971.0,7.270000,0.90,0 days 01:00:14,3614
2061,512321,2021-04-26 16:36:04,2021-04-26 18:08:46,964755.5,10.580000,0.92,0 days 01:32:42,5562


In [142]:
lom_upd = update_lom(lom_train)
lom_upd

,NPLV,VDL_SUM,LOM_SUM
0,510008,25,76200
1,510009,25,78600
2,510010,48,76300
3,510011,25,84100
4,510012,25,76100
...,...,...,...
2058,512318,96,73600
2059,512319,48,76600
2060,512320,65,64200
2061,512321,25,66200


In [143]:
chugun_train

,NPLV,VES,T,SI,MN,S,P,CR,NI,CU,V,TI,DATA_ZAMERA
0,510008,263700.0,1396.0,0.44,0.22,0.023,0.097,0.03,0.01,0.03,0.103,0.084,2021-01-01 03:15:03
1,510009,264500.0,1419.0,0.68,0.20,0.017,0.087,0.02,0.01,0.03,0.084,0.096,2021-01-01 04:23:48
2,510010,263800.0,1384.0,0.56,0.26,0.017,0.096,0.03,0.01,0.03,0.115,0.110,2021-01-01 05:21:40
3,510011,264000.0,1401.0,0.48,0.27,0.018,0.091,0.03,0.01,0.02,0.112,0.110,2021-01-01 06:20:57
4,510012,263300.0,1422.0,0.47,0.23,0.018,0.096,0.02,0.01,0.03,0.083,0.070,2021-01-01 07:23:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,512318,267200.0,1415.0,0.38,0.28,0.019,0.099,0.02,0.01,0.02,0.081,0.060,2021-04-26 12:53:40
2059,512319,266800.0,1405.0,0.50,0.30,0.017,0.104,0.02,0.01,0.02,0.079,0.081,2021-04-26 14:21:57
2060,512320,276100.0,1398.0,0.61,0.31,0.025,0.115,0.03,0.01,0.03,0.086,0.066,2021-04-26 14:33:29
2061,512321,275800.0,1408.0,0.38,0.27,0.021,0.100,0.02,0.01,0.03,0.076,0.060,2021-04-26 15:50:53


In [144]:
plavki_train = plavki_train.drop_duplicates(subset = ['NPLV', 'plavka_STFUT'])
plavki_train

,NPLV,plavka_VR_NACH,plavka_VR_KON,plavka_NMZ,plavka_NAPR_ZAD,plavka_STFUT,plavka_TIPE_FUR,plavka_ST_FURM,plavka_TIPE_GOL,plavka_ST_GOL
0,510008,2021-01-01 03:08:11,2021-01-01 03:51:10,С255,МНЛЗ,971,цилиндрическая,11,5 сопловая,11
1,510009,2021-01-01 04:00:44,2021-01-01 05:07:28,С255,МНЛЗ,972,цилиндрическая,12,5 сопловая,12
2,510010,2021-01-01 05:12:29,2021-01-01 06:00:53,Ст3пс/Э,Изл,973,цилиндрическая,13,5 сопловая,13
3,510011,2021-01-01 06:13:48,2021-01-01 07:08:39,Св-08А.z02,Изл,974,цилиндрическая,14,5 сопловая,14
4,510012,2021-01-01 07:13:44,2021-01-01 08:01:59,SC2M/ЭТ,МНЛС,975,цилиндрическая,15,5 сопловая,15
...,...,...,...,...,...,...,...,...,...,...
2132,512318,2021-04-26 13:04:26,2021-04-26 13:55:50,C071TM.z01/ЭТ,МНЛС,3281,коническая,22,5 сопловая,56
2133,512319,2021-04-26 14:10:20,2021-04-26 15:14:23,C071TM.z01/ЭТ,МНЛС,3282,коническая,23,5 сопловая,57
2134,512320,2021-04-26 15:21:37,2021-04-26 16:16:42,40Х.1,МНЛЗ,3283,коническая,24,5 сопловая,58
2135,512321,2021-04-26 16:22:37,2021-04-26 17:23:37,40Х.1,МНЛЗ,3284,коническая,25,5 сопловая,59


In [145]:
df_train = pd.merge(chugun_train, gas_upd, on = ['NPLV'])
df_train = df_train.merge(chronom_upd, how='right', on = 'NPLV')
df_train = df_train.merge(produv_upd, how='right', on = 'NPLV')
df_train = df_train.merge(lom_upd, how='right', on = 'NPLV')
df_train = df_train.merge(sip_upd, how='right', on = 'NPLV')
df_train = df_train.merge(plavki_train, how='right', on = 'NPLV')

In [148]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2062
Data columns (total 65 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   NPLV                2063 non-null   int64          
 1   VES                 2063 non-null   float64        
 2   T                   2063 non-null   float64        
 3   SI                  2063 non-null   float64        
 4   MN                  2063 non-null   float64        
 5   S                   2063 non-null   float64        
 6   P                   2063 non-null   float64        
 7   CR                  2063 non-null   float64        
 8   NI                  2063 non-null   float64        
 9   CU                  2063 non-null   float64        
 10  V                   2063 non-null   float64        
 11  TI                  2063 non-null   float64        
 12  DATA_ZAMERA         2063 non-null   object         
 13  GAS_START           2063 non-null